In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

{/*
  DO NOT EDIT THIS CELL!!!
  This cell's content is generated automatically by a script. Anything you add
  here will be removed next time the notebook is run. To add new content, create
  a new cell before or after this one.
*/}

<details>
<summary><b>Версії пакетів</b></summary>

Код на цій сторінці був розроблений з використанням наступних вимог.
Ми рекомендуємо використовувати ці версії або новіші.

```
qiskit[all]~=2.3.0
```
</details>
Якщо у вас є набір з $n$ бітів (або кубітів), ви зазвичай позначатимете кожен біт $0
\rightarrow n-1$. Різні програмні забезпечення та ресурси повинні вибрати, як вони впорядковують
ці біти як у пам'яті комп'ютера, так і при відображенні на екрані.

## Конвенції Qiskit

Ось як Qiskit SDK впорядковує біти в різних сценаріях.

### Квантові схеми

Клас `QuantumCircuit` зберігає свої кубіти в списку
(`QuantumCircuit.qubits`). Індекс кубіта в цьому списку визначає
мітку кубіта.

In [1]:
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit import Qubit

qc = QuantumCircuit(2)
qc.qubits[0]  # qubit "0"

Qubit(QuantumRegister(2, "q"), 0)

<Qubit register=(2, "q"), index=0>

### Circuit diagrams

On a circuit diagram, qubit $0$ is the topmost qubit, and qubit $n-1$ the
bottommost qubit. You can change this with the `reverse_bits` argument of
`QuantumCircuit.draw` (see [Change ordering in
Qiskit](#change-ordering-in-qiskit)).

In [2]:
qc.x(1)
qc.draw()

          
q_0: ─────
     ┌───┐
q_1: ┤ X ├
     └───┘

### Діаграми схем
На діаграмі схеми кубіт $0$ є найвищим кубітом, а кубіт $n-1$ —
найнижчим кубітом. Ви можете змінити це за допомогою аргументу `reverse_bits` методу
`QuantumCircuit.draw` (дивіться [Зміна впорядкування в
Qiskit](#change-ordering-in-qiskit)).

In [3]:
from qiskit.primitives import StatevectorSampler as Sampler

qc.measure_all()

job = Sampler().run([qc])
result = job.result()
print(f" > Counts: {result[0].data.meas.get_counts()}")

 > Counts: {'10': 1024}


### Strings

When displaying or interpreting a list of bits (or qubits) as a string, bit
$n-1$ is the leftmost bit, and bit $0$ is the rightmost bit. This is because we
usually write numbers with the most significant digit on the left, and in
Qiskit, bit $n-1$ is interpreted as the most significant bit.

For example, the following cell defines a `Statevector` from a string of
single-qubit states. In this case, qubit $0$ is in state $|+\rangle$, and
qubit $1$ in state $|0\rangle$.

In [4]:
from qiskit.quantum_info import Statevector

sv = Statevector.from_label("0+")
sv.probabilities_dict()

{np.str_('00'): np.float64(0.4999999999999999),
 np.str_('01'): np.float64(0.4999999999999999)}

### Цілі числа
При інтерпретації бітів як числа біт $0$ є найменш значущим бітом, а
біт $n-1$ — найбільш значущим. Це корисно при програмуванні, оскільки кожен біт має
значення $2^\text{label}$ (label — це індекс кубіта в
`QuantumCircuit.qubits`). Наприклад, наступне виконання схеми закінчується
тим, що біт $0$ є `0`, а біт $1$ є `1`. Це інтерпретується як
десяткове ціле число `2` (виміряно з ймовірністю `1.0`).

In [5]:
print(sv[1])  # amplitude of state |01>
print(sv[2])  # amplitude of state |10>

(0.7071067811865475+0j)
0j


### Gates

Each gate in Qiskit can interpret a list of qubits in its own way, but
controlled gates usually follow the convention `(control, target)`.

For example, the following cell adds a controlled-X gate where qubit $0$ is
the control and qubit $1$ is the target.

In [6]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(2)
qc.cx(0, 1)
qc.draw()

          
q_0: ──■──
     ┌─┴─┐
q_1: ┤ X ├
     └───┘

### Рядки
При відображенні або інтерпретації списку бітів (або кубітів) як рядка біт
$n-1$ є крайнім лівим бітом, а біт $0$ — крайнім правим бітом. Це тому, що ми
зазвичай записуємо числа з найбільш значущою цифрою зліва, а в
Qiskit біт $n-1$ інтерпретується як найбільш значущий біт.

Наприклад, наступна комірка визначає `Statevector` з рядка
станів одного кубіта. У цьому випадку кубіт $0$ знаходиться в стані $|+\rangle$, а
кубіт $1$ — в стані $|0\rangle$.

In [7]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(2)
qc.x(0)
qc.draw(reverse_bits=True)

          
q_1: ─────
     ┌───┐
q_0: ┤ X ├
     └───┘

You can use the `reverse_bits` method to return a new circuit with the
qubits' labels reversed (this does not mutate the original circuit).

In [8]:
qc.reverse_bits().draw()

          
q_0: ─────
     ┌───┐
q_1: ┤ X ├
     └───┘

Це іноді викликає плутанину при інтерпретації рядка бітів, оскільки Ви
можете очікувати, що крайній лівий біт буде бітом $0$, тоді як він зазвичай представляє біт
$n-1$.

### Матриці вектора стану
При представленні вектора стану як списку комплексних чисел (амплітуд),
Qiskit впорядковує ці амплітуди так, що амплітуда за індексом $x$ представляє
обчислювальний базисний стан $|x\rangle$.